In [10]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

[autoreload of numpy.core.overrides failed: Traceback (most recent call last):
  File "c:\Users\chanc\anaconda3\envs\tempYellowTaxi\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\chanc\anaconda3\envs\tempYellowTaxi\Lib\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "c:\Users\chanc\anaconda3\envs\tempYellowTaxi\Lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "c:\Users\chanc\anaconda3\envs\tempYellowTaxi\Lib\site-packages\numpy\core\overrides.py", line 6, in <module>
    from numpy.core._multiarray_umath import (
ImportError: cannot import name 'implement_array_fun

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [14]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)

2025-03-04 16:50:58,324 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 16:50:58,334 INFO: Initializing external client
2025-03-04 16:50:58,335 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 16:50:58,884 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214703
Fetching data from 2025-02-03 21:50:58.324653+00:00 to 2025-03-04 20:50:58.324653+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.88s) 


In [15]:
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

TypeError: int() argument must be a string, a bytes-like object or a real number, not '_NoValueType'

In [5]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-04 15:39:51,081 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 15:39:51,081 INFO: Initializing external client
2025-03-04 15:39:51,081 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 15:39:51,667 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214703


In [6]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [7]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,216,1.0,2025-03-04 21:00:00+00:00
1,217,0.0,2025-03-04 21:00:00+00:00
2,255,1.0,2025-03-04 21:00:00+00:00
3,21,0.0,2025-03-04 21:00:00+00:00
4,171,0.0,2025-03-04 21:00:00+00:00
...,...,...,...
246,208,0.0,2025-03-04 21:00:00+00:00
247,114,79.0,2025-03-04 21:00:00+00:00
248,123,0.0,2025-03-04 21:00:00+00:00
249,202,0.0,2025-03-04 21:00:00+00:00


In [8]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-03-04 15:39:53,447 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 15:39:53,450 INFO: Initializing external client
2025-03-04 15:39:53,450 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 15:39:54,041 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214703


In [9]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1214703/fs/1202337/fg/1403495


Uploading Dataframe: 100.00% |██████████| Rows 251/251 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214703/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)